# Decision Trees

In this section, we explore the use of decision trees to model the income data. In particular, we will focus on ensemble methods such as random forests. However, we also explore single classification trees in order to build an understanding for tree-based methods. The document is organised as follows:

**Table of Contents**:
- [1. Introduction to Trees](#1-introduction-to-trees)
    - [1.1. A Brief History](#11-a-brief-history)

- [2. Mathematical Details](#2-mathematical-details)
    - [2.1. Defining the Set of Splits](#21-defining-the-set-of-splits)
    - [2.2. Choosing a Split](#22-choosing-a-split)
    - [2.3. Tree Selection](#23-tree-selection)
    - [2.4. Ensemble Methods](#24-ensemble-methods)

- [3. Upcoming Content](#3-upcoming-content)

- [4. References](#4-references)


# 1. Introduction to Trees

## 1.1 A Brief History
The classification tree is `a child of the computer age' [1] and has gained popularity due to advances in technology. Tree-based methods are non-parametric and first originated in the 1960s through work in the social sciences by Morgan and Sonquist [1]. Several researchers extended this methodology, and in the 1980s Breiman et al., published the seminal CART (Classification and Regression Trees) program. While other methods such as the C4.5 program (due to Quinlan (1993)) and the QUEST method (Loh and Shih (1997)) exist, we focus on the CART methodology in this text. 

Single trees unfortunately have a high variance [4]. To improve on their performance, ensemble methods were developed. The earliest of these methods originated in the 1990s [2, 3]. Particular ensemble methods that we use here are random forests (published by Breiman in 2001 [4]) and XGBoost [5]. These methods use a group of trees, where each tree votes for the classification of an object. 

Next, we give details on how trees work. This explanation is aimed at the practitioner and we aim to elucidate the reader on the different arguments that can be set in the functions we will use. Mathematical enthusiasts can refer to [1, 4, 5] in the references for further details.

# 2. Mathematical Details

Tree-based methods work by recursively partitioning the feature space $\mathcal{X}$ into subsets $A_1, \ldots, A_J$, such that if $\boldsymbol{x} \in A_j$ then the predicted class is $j$. To do this, we use recursive binary splitting. That is, we repeatedly split subsets of $\mathcal{X}$ into two subsets, starting with $\mathcal{X}$ itself, until a rule for termination is met. 

The subsets obtained at the end of this process are called leaf nodes. The 3 key steps to understand are:
- what splits do we consider?
- how do we choose a split? 
- how do we select the size of a tree?

> **Note**: Here, we consider binary splits where each split produces two subsets. However, binary splitting is not the only method that can be used to create a tree. Some methods allow subsets of $\mathcal{X}$ to be split into more than two subsets in a single step. However, we should note that using binary splits is not restrictive, since the partition of $\mathcal{X}$ created by any other type of tree structure can also be created using a binary tree [3].

### 2.1 Defining the Set of Splits
Many implementations of trees consider univariate splits. We ask questions based on single features: `is $x_i \in A$?` and send data answering `No` one way and those answering `Yes` the other way. For example, we may ask `is colour = red?` or `is size of tumour > 54?`. While linear combinations of numerical features (and also boolean combinations of features) can be considered, this considerably increases computation time. This is discussed further in [Section 5.2.2, 5.2.3, 1] but we do not pursue this further for simplicity.

### 2.2 Choosing a Split
Once we have defined the set of splits, we need to choose a split. To do this, we need to evaluate the `quality' of a split. We would like a function $\phi$ that captures our intuition about splits:
- the worst splits are those which lead to a uniform distribution of classes -- these splits did not separate out the classes and lead to `impure' nodes.
- the best splits are those which lead to a single class in a leaf node -- these splits created `purer' nodes.
- the measure of quality should not depend on how we label the classes.

These lead us to consider **impurity functions**, namely the Gini index $\phi_{\text{Gini}}$ and the entropy function $\phi_{\text{Entropy}}$. These are
\begin{equation}
    \phi_{\text{Gini}}(\boldsymbol{p}) = 1 - \sum_{j =1}^2 p_j^2; \quad \quad \phi_\text{Entropy}(\boldsymbol{p}) = -\sum_{j=1}^2 p_j \log p_j, 
\end{equation}

where $\boldsymbol{p} = (p_1, p_2)$ are the proportions of classes 1 and 2 (at a particular node). These functions have maxima at $p_1 = 0.5$ and therefore the worst splits are the uniform splits. The functions have minima at $p_1 = 0, 1$ and are invariant to permutations of $(p_1,p_2)$. They therefore capture our intuitive notion in a mathematical way. Gini punishes large absolute-value mistakes whilst information
punishes large log-scale mistakes [6]. We choose the split which leads to the largest reduction in impurity. This is but one approach to splitting and others may be found in [Chapter 4, 1]. 

We can stop splitting when there are fewer than $N_{min}$ samples in a node (where $N_{min}$ can be decided by the user) or when no significant reduction in impurity is possible (`significant' being decided by the user). The predicted class in the leaf node can be chosen to be the class that minimises a loss (if imbalanced classes) or simply the majority vote (if balanced classes).

### 2.3 Tree Selection
Note that the split termination procedure above is arbitrary. It can lead to termination which is too late, leading to overly large trees which are harder to interpret. It can also lead to too early termination, where we miss out on good later splits. We will instead use an approach called cost-complexity `pruning', where we define a cost (or loss) of a tree and penalise it by the complexity. The term pruning refers to cutting off branches from a tree. 

This leads to minimising a quantity $$\text{Loss}(T) = \text{Cost}(T) + \alpha |T|, $$ where $|T|$ is the number of leaf nodes. This can be done effectively using **weakest-link pruning** [Section 3.3, 1]. We start with the largest tree possible (such that every point is in its own leaf node or such that each leaf node has less than say 5 samples). We then start from a penalty of $\alpha = 0$ where we cut off nothing. We then find the "weakest" branch to cut off as we increase $\alpha$. Cuts only occur at discrete $\alpha$ and the tree is the same in between jump points. This is described in [Section 3.3, 1]. We get a sequence $\alpha_1, \ldots, \alpha_k$. Increasing $\alpha$ beyond $\alpha_k$ produces no further change in the tree as it leads to a tree with no splits. 

This actually allows us to find the optimal subtree just by considering this finite sequence [Section 4.1, 4.2, 7]. We can use cross-validation to find the optimal value of $\alpha$.

### 2.4 Ensemble Methods

Ensemble methods work by using the "wisdom of the crowd". In particular, we use a group of classifiers which can each have a (weighted) vote on the predicted class of an object. This leads to smoother decision boundaries and better performance. Ensemble methods are found to perform very well in practice. In particular, the authors of random forests make "grand claims about the success of random forests:
most accurate, most interpretable, and the like" [p. 590, 8]. On the other hand, XGBoost is described as “an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable” [5].

The theory of ensemble methods is complicated, and we direct the reader to the references and to later sections for further details.

# 3. Upcoming Content

In the next section, we will look at the issues of missingness and imbalance in the data. For computational ease, we will only consider classification trees until we obtain an understanding of how to treat missingness and imbalance. Once we have done this, the succeeding sections will look at ensembles to hopefully obtain cutting edge performance.


# 4. References

[1] Breiman, Leo. Classification and regression trees. Routledge, 2017.

[2] Loh, Wei‐Yin. "Fifty years of classification and regression trees." International Statistical Review 82.3 (2014): 329-348.

[3] Sutton, Clifton D. "Classification and regression trees, bagging, and boosting." Handbook of statistics 24 (2005): 303-329.

[4] Breiman, Leo. "Random forests." Machine learning 45 (2001): 5-32.

[5] Online tutorials on XGBoost and Gradient Boosting: https://xgboost.readthedocs.io/en/latest/ and https://thedatascientist.com/gradient-boosted-trees-python/

[6] Lecture Notes on Random Forests: https://dsbristol.github.io/dst/assets/slides/06.1-RandomForests.pdf

[7] Atkinson, Elizabeth J., and Terry M. Therneau. "An introduction to recursive partitioning using the RPART routines." Rochester: Mayo Foundation 2000 (2000).

[8] Hastie, Trevor, et al. The elements of statistical learning: data mining, inference, and prediction. Vol. 2. New York: springer, 2009.